In [38]:
import hail as hl

In [39]:
new_vcf=hl.read_matrix_table("/home/dpiscia/RDConnect/vcfLoader/output/7.0.0/annotatedVEPdbnSFPCaddClinvarExGnomad/variants19.ht")

In [5]:
new_vcf.count()

(654768, 1058)

In [15]:
654768*1058

692744544

In [34]:
vcf=new_vcf.filter_entries((new_vcf.sample.gt=="0/1") | (new_vcf.sample.gt=="1/1") )

In [35]:
vcf =  vcf.annotate_rows(ref=vcf.alleles[0],
                       alt=vcf.alleles[1],
                       pos=vcf.locus.position,
                       indel=hl.cond((hl.len(vcf.alleles[0]) != (hl.len(vcf.alleles[1]))) | (hl.len(vcf.alleles[0]) != 1) | (hl.len(vcf.alleles[0]) != 1), True, False),
                       samples_germline=hl.filter(lambda x: (x.dp > 8) & (x.gq > 20),hl.agg.collect(vcf.sample)))

In [21]:
#vcf.explode_rows(vcf.samples_germline).count()

(16504112, 1058)

In [16]:
11050072/692744544

0.015951149807973082

In [22]:
16504112/692744544

0.023824239603105413

In [36]:
vcf=vcf.explode_rows(vcf.samples_germline).drop("sample") \
        .rows()

In [ ]:
from pyspark.sql.functions import lit
variants = vcf.to_spark().withColumn("chrom",lit(19))
variants.write.format("org.elasticsearch.spark.sql").save("rdconnect_12/7.0.0", mode='append')

In [122]:
new_vcf_ann.filter_cols(new_vcf_ann.nr==10).count_cols()

2019-06-18 11:30:51 Hail: INFO: Coerced sorted dataset


58

In [132]:
def truncateAt(hl, n, p):
    """ Formats a input number to 'p' decimals
        :param Hailcontext hl: The Hail context
        :param String n: Number to format
        :param String p: Decimal precision
    """
    return hl.float(hl.int((10 ** hl.int(p) * n))) / (10 ** hl.int(p))

In [9]:
new_vcf.explode_rows(new_vcf.sample)

2019-06-20 11:24:50 Hail: ERROR: scope violation: 'MatrixTable.explode_rows' expects an expression indexed by ['row']
    Found indices ['column', 'row'], with unexpected indices ['column']. Invalid fields:
        'sample' (indices ['column', 'row'])


ExpressionException: scope violation: 'MatrixTable.explode_rows' expects an expression indexed by ['row']
    Found indices ['column', 'row'], with unexpected indices ['column']. Invalid fields:
        'sample' (indices ['column', 'row'])

In [162]:
MIN_DP=8
from pyspark.sql.functions import lit

for group in range(0,datamap.get("nr")//100+1):
    print(group)
    vcf=new_vcf_ann.filter_cols(new_vcf_ann.nr==group)
    vcf =  vcf.annotate_rows(ref=vcf.alleles[0],
                       alt=vcf.alleles[1],
                       pos=vcf.locus.position,
                       indel=hl.cond((hl.len(vcf.alleles[0]) != (hl.len(vcf.alleles[1]))) | (hl.len(vcf.alleles[0]) != 1) | (hl.len(vcf.alleles[0]) != 1), True, False),
                       samples_germline=hl.filter(lambda x: (x.dp > 8) & (x.gq > 20),hl.agg.collect(vcf.sample)))
    vcf = vcf.annotate_rows(freqIntGermline = hl.cond((hl.len(vcf.samples_germline) > 0) | (hl.len(hl.filter(lambda x: x.dp > MIN_DP,vcf.samples_germline)) > 0),
        truncateAt(hl,hl.sum(hl.map(lambda x: x.gtInt.unphased_diploid_gt_index(),vcf.samples_germline))/hl.sum(hl.map(lambda x: 2,hl.filter(lambda x: x.dp > 8,vcf.samples_germline))),"6"), 0.0)) \
        .drop("sample") \
        .rows()
    variants = vcf.to_spark().withColumn("chrom",lit(19))
    variants.write.format("org.elasticsearch.spark.sql").save("rdconnect_10/7.0.0", mode='append')
    
    
    
    
    

0
1
2
3
4
5
6
7
8
9
10


In [147]:
vcf_filtered=vcf.filter_cols(vcf.nr==1)

In [ ]:
drop.sample germline gtint

In [150]:
vcf_filtered.annotate_rows(gt= vcf_filtered.alleles[0])

In [106]:
new_vcf_ann = new_vcf.annotate_rows(rsa=hl.eval(annotationsDbSNP()))

In [107]:
new_vcf_ann.row.take(1)

[Struct(locus=Locus(contig=19, position=60251, reference_genome=GRCh37), alleles=['G', 'A'], a_index=1, was_split=False, effs=[Struct(gene_name='WASH5P', effect_impact='MODIFIER', transcript_id='ENST00000592209', effect='intron_variant,non_coding_transcript_variant', gene_id='ENSG00000225373', functional_class='transcript', amino_acid_length='', codon_change='n.406-89C>T', amino_acid_change=None, exon_rank=None, transcript_biotype='processed_transcript', gene_coding=None), Struct(gene_name='WASH5P', effect_impact='MODIFIER', transcript_id='ENST00000606546', effect='intron_variant,non_coding_transcript_variant', gene_id='ENSG00000225373', functional_class='transcript', amino_acid_length='', codon_change='n.179-89C>T', amino_acid_change=None, exon_rank=None, transcript_biotype='processed_transcript', gene_coding=None), Struct(gene_name='WASH5P', effect_impact='MODIFIER', transcript_id='ENST00000606728', effect='downstream_gene_variant', gene_id='ENSG00000225373', functional_class='transc

In [60]:
new_vcf_ann.filter_cols(new_vcf_ann.nr==0).count_cols()

2019-06-18 10:36:54 Hail: INFO: Coerced sorted dataset


1058

In [21]:
for group in range(0,datamap['nr']//100):
    print(group)

0
1
2
3
4
5
6
7
8
9
